NESTML active dendrite third-factor STDP synapse
==========================================

In this tutorial, a neuron and synapse model are defined in NESTML that are subsequently used in a network to perform learning, prediction and replay of sequences of items, such as letters, images or sounds [1].

<a name="introduction"></a>

Introduction
------------


In [1]:
%matplotlib inline

from typing import List, Optional

import matplotlib as mpl

mpl.rcParams['axes.formatter.useoffset'] = False
mpl.rcParams['axes.grid'] = True
mpl.rcParams['grid.color'] = 'k'
mpl.rcParams['grid.linestyle'] = ':'
mpl.rcParams['grid.linewidth'] = 0.5
mpl.rcParams['figure.dpi'] = 120
mpl.rcParams['figure.figsize'] = [8., 3.]
# plt.rcParams['font.size'] = 8
# plt.rcParams['legend.fontsize']= 6
# plt.rcParams['figure.figsize'] = fig_size
# plt.rcParams['savefig.dpi'] = 300
# plt.rcParams['font.family'] = 'sans-serif'
# plt.rcParams['lines.linewidth'] = 1
# plt.rcParams['text.usetex'] = False


import matplotlib.pyplot as plt
import nest
import numpy as np
import os
import random
import re

import parameters as para

import shtm
import shtm.model
import shtm.helper
import copy
import time

from pynestml.codegeneration.nest_code_generator_utils import NESTCodeGeneratorUtils
from pynestml.codegeneration.nest_tools import NESTTools

/home/charl/.local/lib/python3.11/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "



              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.5.0-rc1
 Built: Feb 21 2024 09:04:08

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.



## Generating code with NESTML

We will take a simple current-based integrate-and-fire model with alpha-shaped postsynaptic response kernels (``iaf_psc_alpha``) as the basis for our modifications. First, let's take a look at this base neuron without any modifications.

We will use a helper function to generate the C++ code for the models, build it as a NEST extension module, and load the module into the kernel. Because NEST does not support un- or reloading of modules at the time of writing, we implement a workaround that appends a unique number to the name of each generated model, for example, "iaf_psc_alpha_3cc945f". The resulting neuron model name is returned by the function, so we do not have to think about these internals.

In [ ]:
%pdb
# codegen_opts = {"neuron_synapse_pairs": [{"neuron": "iaf_psc_exp_dend",
#                                           "synapse": "third_factor_stdp_synapse",
#                                           "post_ports": ["post_spikes",
#       http://localhost:8888/notebooks/doc/tutorials/sequences/sequences.ipynb#            
#["I_post_dend", "I_dend"]]}]}

# if not NESTTools.detect_nest_version().startswith("v2"):
#     codegen_opts["neuron_parent_class"] = "StructuralPlasticityNode"
#     codegen_opts["neuron_parent_class_include"] = "structural_plasticity_node.h"

# generate the "jit" model (co-generated neuron and synapse), that does not rely on ArchivingNode
# files = [os.path.join("models", "neurons", "iaf_psc_exp_dend_neuron.nestml"),
#          os.path.join("models", "synapses", "third_factor_stdp_synapse.nestml")]
# input_path = [os.path.realpath(os.path.join(os.path.dirname(__file__), os.path.join(
#     os.pardir, os.pardir, s))) for s in files]
# generate_nest_target(input_path=input_path,
#                      target_path="/tmp/nestml-jit",
#                      logging_level="INFO",
#                      module_name="nestml_jit_module",
#                      codegen_opts=codegen_opts)
#nest.Install("nestml_jit_module")

# generate and build code

module_name, neuron_model_name, synapse_model_name = \
    NESTCodeGeneratorUtils.generate_code_for("../../../doc/tutorials/sequences/iaf_psc_exp_nonlineardendrite_neuron.nestml",
                                             "../../../models/synapses/stdsp_synapse.nestml",
                                             logging_level="DEBUG",
                                             post_ports=["post_spikes", ["dAP_trace", "dAP_trace"]])
"""

module_name = "nestml_98cfb66389b04bd6af313317a8b164a5_module"
neuron_model_name = "iaf_psc_exp_nonlineardendrite98cfb66389b04bd6af313317a8b164a5_neuron_nestml__with_stdsp98cfb66389b04bd6af313317a8b164a5_synapse_nestml"
synapse_model_name = "stdsp98cfb66389b04bd6af313317a8b164a5_synapse_nestml__with_iaf_psc_exp_nonlineardendrite98cfb66389b04bd6af313317a8b164a5_neuron_nestml"
"""


"""# JUST THE NEURON MODEL
module_name, neuron_model_name = \
    NESTCodeGeneratorUtils.generate_code_for("../../../doc/tutorials/sequences/iaf_psc_exp_nonlineardendrite_neuron.nestml",
                                             logging_level="DEBUG")"""


# load dynamic library (NEST extension module) into NEST kernel
nest.Install(module_name)

Automatic pdb calling has been turned ON


NESTErrors.DynamicModuleManagementError: DynamicModuleManagementError in SLI function Install: Module 'nestml_98cfb66389b04bd6af313317a8b164a5_module' could not be opened.
The dynamic loader returned the following error: 'file not found'.

Please check LD_LIBRARY_PATH (OSX: DYLD_LIBRARY_PATH)!

> /home/charl/julich/nest-simulator-install/lib/python3.11/site-packages/nest/ll_api.py(104)catching_sli_run()
    102 
    103         exceptionCls = getattr(kernel.NESTErrors, errorname)
--> 104         raise exceptionCls(commandname, message)
    105 
    106 



Now, the NESTML model is ready to be used in a simulation.

## Helper functions

In [ ]:
def psp_max_2_psc_max(psp_max, tau_m, tau_s, R_m):
    """Compute the PSC amplitude (pA) injected to get a certain PSP maximum (mV) for LIF with exponential PSCs

    Parameters
    ----------
    psp_max: float
             Maximum postsynaptic pontential
    tau_m:   float
             Membrane time constant (ms).
    tau_s:   float
             Synaptic time constant (ms).
    R_m:     float
             Membrane resistance (Gohm).

    Returns
    -------
    float
        PSC amplitude (pA).
    """

    return psp_max / (
            R_m * tau_s / (tau_s - tau_m) * (
            (tau_m / tau_s) ** (-tau_m / (tau_m - tau_s)) -
            (tau_m / tau_s) ** (-tau_s / (tau_m - tau_s))
    )
    )


In [ ]:
DELAY = 0.1

p = para.ParameterSpace({})








p['dt'] = 0.1                                  # simulation time resolution (ms)
p['print_simulation_progress'] = False         # print the time progress.

# neuron parameters of the excitatory neurons
p['soma_model'] = neuron_model_name
p['soma_params'] = {}
p['soma_params']['C_m'] = 250.        # membrane capacitance (pF)
p['soma_params']['E_L'] = 0.          # resting membrane potential (mV)
# p['soma_params']['I_e'] = 0.        # external DC currents (pA)
p['soma_params']['V_m'] = 0.          # initial potential (mV)
p['soma_params']['V_reset'] = 0.      # reset potential (mV)
p['soma_params']['V_th'] = 20.        # spike threshold (mV)
p['soma_params']['t_ref'] = 10.       # refractory period
p['soma_params']['tau_m'] = 10.       # membrane time constant (ms)
p['soma_params']['tau_syn1'] = 2.     # synaptic time constant: external input (receptor 1)
p['soma_params']['tau_syn2'] = 5.     # synaptic time constant: dendrtic input (receptor 2)
p['soma_params']['tau_syn3'] = 1.     # synaptic time constant: inhibitory input (receptor 3)
# dendritic action potential
p['soma_params']['I_p'] = 200. # current clamp value for I_dAP during a dendritic action potenti
p['soma_params']['tau_dAP'] = 60.       # time window over which the dendritic current clamp is active
p['soma_params']['theta_dAP'] = 59.        # current threshold for a dendritic action potential

p['soma_params']['I_dend_incr'] = 2.71 / 10e-3


p['fixed_somatic_delay'] = 2          # this is an approximate time of how long it takes the soma to fire
                                      # upon receiving an external stimulus 

    

# neuron parameters for the inhibitory neuron
p['inhibit_model'] = 'iaf_psc_exp'
p['inhibit_params'] = {}
p['inhibit_params']['C_m'] = 250.         # membrane capacitance (pF)
p['inhibit_params']['E_L'] = 0.           # resting membrane potential (mV)
p['inhibit_params']['I_e'] = 0.           # external DC currents (pA)
p['inhibit_params']['V_m'] = 0.           # initial potential (mV)
p['inhibit_params']['V_reset'] = 0.       # reset potential (mV)
p['inhibit_params']['V_th'] = 15.         # spike threshold (mV)
p['inhibit_params']['t_ref'] = 2.0        # refractory period
p['inhibit_params']['tau_m'] = 5.         # membrane time constant (ms)
p['inhibit_params']['tau_syn_ex'] = .5    # synaptic time constant of an excitatory input (ms) 
p['inhibit_params']['tau_syn_in'] = 1.65  # synaptic time constant of an inhibitory input (ms)

    
    
    
# synaptic parameters
p['J_EX_psp'] = 1.1 * p['soma_params']['V_th']     # somatic PSP as a response to an external input
p['J_IE_psp'] = 1.2 * p['inhibit_params']['V_th']  # inhibitory PSP as a response to an input from E neuron
p['J_EI_psp'] = -2 * p['soma_params']['V_th']      # somatic PSP as a response to an inhibitory input
p['convergence'] = 5




# parameters for setting up the network  
p['M'] = 6                   # number of subpopulations
p['n_E'] = 150               # number of excitatory neurons per subpopulation
p['n_I'] = 1                 # number of inhibitory neurons per subpopulation
p['L'] = 1                   # number of subpopulations that represents one sequence element
p['pattern_size'] = 20       # sparse set of active neurons per subpopulation

# connection details
p['rule'] = 'fixed_indegree'                          
p['connection_prob'] = 0.2

# neuron parameters of the excitatory neurons
p['soma_model'] = neuron_model_name
p['soma_params'] = {}
p['soma_params']['C_m'] = 250.        # membrane capacitance (pF)
p['soma_params']['E_L'] = 0.          # resting membrane potential (mV)
# p['soma_params']['I_e'] = 0.        # external DC currents (pA)
p['soma_params']['V_m'] = 0.          # initial potential (mV)
p['soma_params']['V_reset'] = 0.      # reset potential (mV)
p['soma_params']['V_th'] = 20.        # spike threshold (mV)
p['soma_params']['t_ref'] = 10.       # refractory period
p['soma_params']['tau_m'] = 10.       # membrane time constant (ms)
p['soma_params']['tau_syn1'] = 2.     # synaptic time constant: external input (receptor 1)
p['soma_params']['tau_syn2'] = 5.     # synaptic time constant: dendrtic input (receptor 2)
p['soma_params']['tau_syn3'] = 1.     # synaptic time constant: inhibitory input (receptor 3)
# dendritic action potential
p['soma_params']['I_p'] = 200. # current clamp value for I_dAP during a dendritic action potenti
p['soma_params']['tau_dAP'] = 60.       # time window over which the dendritic current clamp is active
p['soma_params']['theta_dAP'] = 59.        # current threshold for a dendritic action potential
p['fixed_somatic_delay'] = 2          # this is an approximate time of how long it takes the soma to fire
                                      # upon receiving an external stimulus 

# synaptic parameters
p['J_EX_psp'] = 1.1 * p['soma_params']['V_th']     # somatic PSP as a response to an external input
p['J_EI_psp'] = -2 * p['soma_params']['V_th']      # somatic PSP as a response to an inhibitory input
p['convergence'] = 5

# parameters for ee synapses (stdsp)
p['syn_dict_ee'] = {}
p['permanence_min'] = 0.
p['permanence_max'] = 8.
p['calibration'] = 0.
p['syn_dict_ee']['weight'] = 0.01                    # synaptic weight
p['syn_dict_ee']['synapse_model'] = synapse_model_name  # synapse model
p['syn_dict_ee']['permanence_threshold'] = 10.                    # synapse maturity threshold
p['syn_dict_ee']['tau_pre_trace'] = 20.                   # plasticity time constant (potentiation)
p['syn_dict_ee']['delay'] = 2.                       # dendritic delay 
p['syn_dict_ee']['receptor_type'] = 2                # receptor corresponding to the dendritic input
p['syn_dict_ee']['lambda_plus'] = 0.08               # potentiation rate
p['syn_dict_ee']['zt'] = 1.                          # target dAP trace
p['syn_dict_ee']['lambda_h'] = 0.014                 # homeostasis rate
p['syn_dict_ee']['Wmax'] = 1.1 * p['soma_params']['theta_dAP'] / p['convergence']   # Maximum allowed weight
p['syn_dict_ee']['permanence_max'] = 20.                       # Maximum allowed permanence
p['syn_dict_ee']['permanence_min'] = 1.                        # Minimum allowed permanence
p['syn_dict_ee']['lambda_minus'] = 0.0015            # depression rate
p['syn_dict_ee']['dt_min'] = -4.                     # minimum time lag of the STDP window
p['inh_factor'] = 7.

# parameters of EX synapses (external to soma of E neurons)
p['conn_dict_ex'] = {}
p['syn_dict_ex'] = {}
p['syn_dict_ex']['receptor_type'] = 1                    # receptor corresponding to external input
p['syn_dict_ex']['delay'] = DELAY                        # dendritic delay
p['conn_dict_ex']['rule'] = 'all_to_all'                 # connection rule

# parameters of EdX synapses (external to dendrite of E neurons) 
p['conn_dict_edx'] = {}
p['syn_dict_edx'] = {}
p['syn_dict_edx']['receptor_type'] = 2                    # receptor corresponding to the dendritic input
p['syn_dict_edx']['delay'] = DELAY                        # dendritic delay
p['syn_dict_edx']['weight'] = 1.4 * p['soma_params']['theta_dAP']
p['conn_dict_edx']['rule'] = 'fixed_outdegree'            # connection rule
p['conn_dict_edx']['outdegree'] = p['pattern_size'] + 1   # outdegree

# parameters for IE synapses 
p['syn_dict_ie'] = {}
p['conn_dict_ie'] = {}
p['syn_dict_ie']['synapse_model'] = 'static_synapse'     # synapse model
p['syn_dict_ie']['delay'] = DELAY                        # dendritic delay
p['conn_dict_ie']['rule'] = 'fixed_indegree'             # connection rule
p['conn_dict_ie']['indegree'] = 5                        # indegree 

# parameters for EI synapses
p['syn_dict_ei'] = {}
p['conn_dict_ei'] = {}
p['syn_dict_ei']['synapse_model'] = 'static_synapse'     # synapse model
p['syn_dict_ei']['delay'] = DELAY                        # dendritic delay
p['syn_dict_ei']['receptor_type'] = 3                    # receptor corresponding to the inhibitory input  
p['conn_dict_ei']['rule'] = 'fixed_indegree'             # connection rule
p['conn_dict_ei']['indegree'] = 20                       # indegree

# stimulus parameters
p['DeltaT'] = 40.                     # inter-stimulus interval
p['excitation_start'] = 30.           # time at which the external stimulation begins
p['time_dend_to_somatic'] = 20.       # time between the dAP activation and the somatic activation (only used if sparse_first_char is True)   
p['DeltaT_cue'] = 80.                 # inter-cue interval during replay

# simulation parameters 
p['dt'] = 0.1                                  # simulation time resolution (ms)
p['overwrite_files'] = True                    # if True, data will be overwritten,
                                               # if False, a NESTError is raised if the files already exist
p['seed'] = para.ParameterRange([1,2,3,4,5])   # seed for NEST
p['print_simulation_progress'] = False         # print the time progress.
p['pad_time'] = 5.
p['idend_recording_interval'] = 10 * p['dt']   # dendritic current recording resolution
p['idend_record_time'] = 8.                    # time interval after the external stimulation at which the dendritic current is recorded
p['evaluate_performance'] = True               # if turned on, we monitor the dendritic current at a certain time steps
                                               # during the simulation. This then is used for the prediction performance assessment
p['evaluate_replay'] = False                     
p['record_idend_last_episode'] = True          # used for debugging, if turned on we record the dendritic current of all neurons
                                               # this can consume too much memory
p['store_connections'] = True              
p['load_connections'] = False
p['sparse_first_char'] = False                 # if turned on, the dAP of a subset of neurons in the subpopulation representing 
                                               # first sequence elements is activated externally 
p['active_weight_recorder'] = False            # if turned on, the weights are recorded every presynaptic spike

# task parameters
p['task'] = {}
p['task']['task_name'] = 'hard_coded'          # name of the task
p['task']['task_type'] = 1                     # this chooses between three hard coded sequence sets (see ./utils.py)
p['task']['vocab_size'] = 6                   # vocabulary size
p['task']['seed'] = 111                        # seed number
p['task']['store_training_data'] = True        # if turned on, the sequence set is stored in directory defined in dict data_path
if p['task']['task_name'] != 'hard_coded':
    p['task']['num_sequences'] = 2             # number of sequences per sequence set
    p['task']['num_sub_seq'] = 2               # if task_name == 'high_order', 
                                               # it sets the number of sequences with same shared subsequence
    p['task']['length_sequence'] = 6           # number of elements per sequence
    p['task']['replace'] = False               # random choice of characters with replacement

# setup the training loop  
p['learning_episodes'] = 3#85                     # total number of training episodes ('repetitions of the sequence sets')
p['episodes_to_testing'] = 1                   # number of episodes after which we measure the prediction perfomance

In [ ]:
params = p
params['R_m_soma'] = params['soma_params']['tau_m'] / params['soma_params']['C_m']
params['R_m_inhibit'] = params['inhibit_params']['tau_m'] / params['inhibit_params']['C_m']
params['syn_dict_ex']['weight'] = psp_max_2_psc_max(params['J_EX_psp'], 
                                                           params['soma_params']['tau_m'], 
                                                           params['soma_params']['tau_syn1'], 
                                                           params['R_m_soma'])
params['syn_dict_ie']['weight'] = psp_max_2_psc_max(params['J_IE_psp'], 
                                                           params['inhibit_params']['tau_m'], 
                                                           params['inhibit_params']['tau_syn_ex'], 
                                                           params['R_m_inhibit'])
params['syn_dict_ei']['weight'] = psp_max_2_psc_max(params['J_EI_psp'], 
                                                           params['soma_params']['tau_m'], 
                                                           params['soma_params']['tau_syn3'], 
                                                           params['R_m_soma'])

soma_excitation_time = 25.
dendrite_excitation_time = 3.

In [ ]:
# experiments/sequence_learning_and_prediction/local_simulation.py

def generate_sequences(params, fname):
    """Generate sequence of elements using three methods:
    1. randomly drawn elements from a vocabulary
    2. sequences with transition matrix
    3. higher order sequences: sequences with shared subsequences
    4. hard coded sequences

    Parameters
    ----------
    params : dict
        dictionary contains task parameters
    fname       : str

    Returns
    -------
    sequences: list
    test_sequences: list
    vocabulary: list
    """

    task_name = params['task_name']
    task_type = params['task_type']
 
    # set of characters used to build the sequences
    vocabulary = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T',
                  'U', 'V', 'W', 'X', 'Y', 'Z'][:params['vocab_size']]
    sequences = []

    # create high order sequences, characters are drawn without replacement
    if task_name == "high_order":

        if (params['num_sequences'] % params['num_sub_seq'] != 0):
            raise ZeroDivisionError(
                'for high order sequences number of sequences needs ("num_sequences") to be divisible by num_sub_seq')

        num_sequences_high_order = int(params['num_sequences'] / params['num_sub_seq'])
        for i in range(num_sequences_high_order):
            characters_sub_seq = copy.copy(vocabulary)
            sub_seq = random.sample(characters_sub_seq, params["length_sequence"] - 2)
            for char in sub_seq:
                characters_sub_seq.remove(char)

            for j in range(params['num_sub_seq']):
                # remove the characters that were chosen for the end and the start of the sequence
                # this is to avoid sequences with adjacent letters of the same kind
                # we will add this feature to the code asap 
                end_char = random.sample(characters_sub_seq, 1)
                characters_sub_seq.remove(end_char[0])

                start_char = random.sample(characters_sub_seq, 1)
                characters_sub_seq.remove(start_char[0])

                sequence = start_char + sub_seq + end_char
                sequences.append(sequence)

                # randomly shuffled characters
    elif task_name == "random":
        sequences = [random.sample(vocabulary, length_seq) for _ in range(params['num_sequences'])]

    # create sequences using matrix transition 
    elif task_name == "structure":
        matrix_transition = defaultdict(list)
        for char in vocabulary:
            x = np.random.choice(2, len(vocabulary), p=[0.2, 0.8])
            matrix_transition[char] = x / sum(x)

        for _ in range(params['num_sequences']):
            sequence = random.sample(vocabulary, 1)
            last_char = sequence[-1]
            for _ in range(length_seq - 1):
                sequence += np.random.choice(vocabulary, 1, p=matrix_transition[last_char])[0]
                last_char = sequence[-1]

            sequences += [sequence]
    else:

        # hard coded sequences 
        if task_type == 1:
            sequences = [['A', 'D', 'B', 'E'], ['F', 'D', 'B', 'C']]
        elif task_type == 2:
            sequences = [['E', 'N', 'D', 'I', 'J'], ['L', 'N', 'D', 'I', 'K'], ['G', 'J', 'M', 'C', 'N'], 
                         ['F', 'J', 'M', 'C', 'I'], ['B', 'C', 'K', 'H', 'I'], ['A', 'C', 'K', 'H', 'F']]
        elif task_type == 3:
            sequences = [['E', 'N', 'D', 'I', 'J'], ['L', 'N', 'D', 'I', 'K'], ['G', 'J', 'M', 'E', 'N'], 
                         ['F', 'J', 'M', 'E', 'I'], ['B', 'C', 'K', 'B', 'I'], ['A', 'C', 'K', 'B', 'F']]
        else:
            sequences = [['A', 'D', 'B', 'G', 'H', 'E'], ['F', 'D', 'B', 'G', 'H', 'C']]

    # test sequences used to measure the accuracy 
    test_sequences = sequences

    fname = 'training_data'
    fname_voc = 'vocabulary'
    print("\nSave training data to %s" % (fname))
    np.save('%s' % ( fname), sequences)
    np.save('%s' % ( fname_voc), vocabulary)

    return sequences, test_sequences, vocabulary

model_instance = None
def generate_reference_data():
    global model_instance
    #############################################################
    # get network and training parameters 
    # ===========================================================
    global p
    PS = copy.deepcopy(p)

    # parameter-set id from command line (submission script)
    PL = shtm.helper.parameter_set_list(PS) 
    params = PL[0]

    # start time 
    time_start = time.time()

    # ###############################################################
    # specify sequences
    # ===============================================================
    sequences, _, vocabulary = generate_sequences(params['task'], PL[0]['label'])

    # ###############################################################
    # create network
    # ===============================================================
    model_instance = shtm.model.Model(params, sequences, vocabulary)
    time_model = time.time()

    model_instance.create()
    time_create = time.time()

    # ###############################################################
    # connect the netwok
    # ===============================================================
    model_instance.connect()
    time_connect = time.time()
    
    # store connections before learning
    if params['store_connections']:
        model_instance.save_connections(fname='ee_connections_before')

    # ###############################################################
    # simulate the network
    # ===============================================================
    model_instance.simulate()
    time_simulate = time.time()

    # store connections after learning
    if params['store_connections']:
        model_instance.save_connections(fname='ee_connections')

    print(
        '\nTimes of Rank {}:\n'.format(
            nest.Rank()) +
        '  Total time:          {:.3f} s\n'.format(
            time_simulate -
            time_start) +
        '  Time to initialize:  {:.3f} s\n'.format(
            time_model -
            time_start) +
        '  Time to create:      {:.3f} s\n'.format(
            time_create -
            time_model) +
        '  Time to connect:     {:.3f} s\n'.format(
            time_connect -
            time_create) +
        '  Time to simulate:    {:.3f} s\n'.format(
            time_simulate -
            time_connect))

    # display prediction performance only for debugging    
    if params['evaluate_performance']:
    
        # print Ic
        #zs = np.array([nest.GetStatus(model_instance.exc_neurons)[i]['z'] for i in range(params['M']*params['n_E'])])
        #id_zs = np.where(zs>0.5)
        #print(zs[id_zs])

        # load spikes from reference data
        somatic_spikes = shtm.helper.load_spike_data(model_instance.data_path, 'somatic_spikes')
        idend_eval = shtm.helper.load_spike_data(model_instance.data_path, 'idend_eval')
        excitation_times = shtm.helper.load_data(model_instance.data_path, 'excitation_times')

        # get recoding times of dendriticAP
        idend_recording_times = shtm.helper.load_data(model_instance.data_path,  'idend_recording_times')
        characters_to_subpopulations = shtm.helper.load_data(model_instance.data_path,  'characters_to_subpopulations')

        seq_avg_errors, seq_avg_false_positives, seq_avg_false_negatives, _ = shtm.helper.compute_prediction_performance(somatic_spikes, idend_eval, idend_recording_times, characters_to_subpopulations, model_instance.sequences, model_instance.params)

        # get number of active neuron for each element in the sequence
        number_elements_per_batch = sum([len(seq) for seq in model_instance.sequences])
        start_time = excitation_times[-number_elements_per_batch] - 5 
        end_time = excitation_times[-1] + 5

        idx_times = np.where((np.array(excitation_times) > start_time) & (np.array(excitation_times) < end_time))  
        excitation_times_sel = np.array(excitation_times)[idx_times]

        num_active_neurons = shtm.helper.number_active_neurons_per_element(model_instance.sequences, somatic_spikes[:,1], somatic_spikes[:,0], excitation_times_sel, params['fixed_somatic_delay'])

        print("\n##### testing sequences with number of somatic spikes ")
        count_false_negatives = 0
        for i, (sequence, seq_counts) in enumerate(zip(model_instance.sequences, num_active_neurons)): 
            seq = ''
            for j, (char, counts) in enumerate(zip(sequence, seq_counts)):
                seq += str(char)+'('+ str(seq_counts[char])+')'.ljust(2)

                if j != 0 and seq_counts[char] > 0.5*params['n_E']:
                    count_false_negatives += 1

            print("sequence %d: %s" % (i, seq))   

        print("False negative counts", count_false_negatives)   

    print("\n### Plasticity parameters")
    print("lambda plus: %0.4f" % params['syn_dict_ee']['lambda_plus'])
    print("lambda homeostasis: %0.4f" % params['syn_dict_ee']['lambda_h'])
    print("lambda minus: %0.4f" % model_instance.params['syn_dict_ee']['lambda_minus']) 
    print("inh factor:", params['inh_factor'])
    print("excitation step %0.1fms" % params['DeltaT']) #30-50  
    print("seed number: %d" % params['seed']) 
    print("number of learning episodes: %d" % params['learning_episodes'])

generate_reference_data()




In [ ]:
# experiments/sequence_learning_and_prediction/prediction_performance_analysis.py

PS = copy.deepcopy(p)
PS_sel = copy.deepcopy(PS)
compute_overlap = True

PL = shtm.helper.parameter_set_list(PS_sel)

# get training data
sequences = shtm.helper.load_data('.', 'training_data')

print("#### sequences used for training ### ")
for i, sequence in enumerate(sequences): 
    seq = '' 
    for char in sequence:
        seq += str(char).ljust(2) 
    print("sequence %d: %s" % (i, seq))

fname = 'prediction_performance'
for cP, params in enumerate(PL):

    data = {}

    # get data path
    # load somatic spikes and dendritic current
    somatic_spikes = shtm.helper.load_spike_data(model_instance.data_path, 'somatic_spikes')
    idend_eval = shtm.helper.load_spike_data(model_instance.data_path,  'idend_eval')

    # load record and excitation times 
    idend_recording_times = shtm.helper.load_data(model_instance.data_path,  'idend_recording_times')
    characters_to_subpopulations = shtm.helper.load_data(model_instance.data_path,  'characters_to_subpopulations')
    excitation_times = shtm.helper.load_data(model_instance.data_path,  'excitation_times')

    # compute prediction performance
    errors, false_positives, false_negatives, num_active_neurons = shtm.helper.compute_prediction_performance(somatic_spikes, idend_eval, idend_recording_times, characters_to_subpopulations, sequences, params)

    if compute_overlap:
        # sequences overlap
        sequences_overlap = shtm.helper.measure_sequences_overlap(sequences, somatic_spikes[:,1], somatic_spikes[:,0], excitation_times, params['fixed_somatic_delay'], params['learning_episodes'])
        data['overlap'] = sequences_overlap

    data['error'] = errors
    data['false_positive'] = false_positives
    data['false_negative'] = false_negatives
    data['rel_active_neurons'] = num_active_neurons/params['n_E']
    data['ep_num'] = params['episodes_to_testing'] * np.arange(int(params['learning_episodes']/params['episodes_to_testing'])+1)

    ep_to_sol = np.where(errors < 0.01)[0] 
    if len(ep_to_sol) == 0:
        print("number of episodes to convergence", params['learning_episodes'])
    else:    
        print("number of episodes to convergence", data['ep_num'][ep_to_sol][0])

    # save data
    np.save("%s" % (fname), data)


In [ ]:
def load_data(path, fname):
    """Load data

    Parameters
    ----------
    path: str
    fname: str

    Returns
    -------
    data: ndarray
    """

    #TODO: this is temporary hack!
    try:
      data = np.load('%s/%s.npy' % (path, fname), allow_pickle=True).item()
    except:
      data = np.load('%s/%s.npy' % (path, fname), allow_pickle=True)

    return data



# get parameters 
PS = p
PS_sel = copy.deepcopy(PS)

params = PS_sel
num_neurons = params['M'] * params['n_E']

# get trained sequences and vocabulary
sequences = shtm.helper.load_data('.', 'training_data')
vocabulary = shtm.helper.load_data('.', 'vocabulary')

print('#### sequences used for training ### ')
for i, sequence in enumerate(sequences):
    seq = ''
    for char in sequence:
        seq += str(char).ljust(2)
    print('sequence %d: %s' % (i, seq))

# load spikes
somatic_spikes = shtm.helper.load_spike_data(model_instance.data_path, 'somatic_spikes')
idend = shtm.helper.load_spike_data(model_instance.data_path, 'idend_last_episode')

# load spike recordings
idend_recording_times = shtm.helper.load_data(model_instance.data_path,  'idend_recording_times')
characters_to_subpopulations = shtm.helper.load_data(model_instance.data_path, 'characters_to_subpopulations')
characters_to_time_excitation = shtm.helper.load_data(model_instance.data_path, 'excitation_times_soma')

# load excitation times
excitation_times = shtm.helper.load_data(model_instance.data_path, 'excitation_times')

# get dendritic AP
idx = np.where((idend[:, 2] > params['soma_params']['theta_dAP']))[0]
dendriticAP_currents = idend[:, 2][idx]
dendriticAP_times = idend[:, 1][idx]
dendriticAP_senders = idend[:, 0][idx]

# organize the characters for plotting purpose
subpopulation_indices = []
chars_per_subpopulation = []
for char in vocabulary:
    # shift the subpopulation indices for plotting purposes 
    char_to_subpopulation_indices = characters_to_subpopulations[char]
    subpopulation_indices.extend(char_to_subpopulation_indices)

    chars_per_subpopulation.extend(char * len(characters_to_subpopulations[char]))

shifted_subpopulation_indices = np.array(subpopulation_indices) + 0.5

# ####################################################
# plotting routing
# ----------------------------------------------------

# plot settings 
fig_size = (5.2, 5.7)
plt.rcParams['font.size'] = 8
plt.rcParams['legend.fontsize'] = 6
plt.rcParams['figure.figsize'] = fig_size
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['text.usetex'] = False

panel_label_pos = (-0.14,0.5)
panel_labels = ['B', 'D', 'F']
color_soma_spike = '#DB2763'
color_dendrite_spike = '#00B4BE' 
fc_bg = '#dcdcdc'
fraction_active = 3
delta_time = 5.
ymin = -0.1
ymax = 2
xmin = 0
master_file_name = 'network_activity'

# set up the figure frame
fig = plt.figure()
gs = mpl.gridspec.GridSpec(6, 2, height_ratios=[3, 15, 3, 15, 3, 15], bottom=0.07, right=0.95, top=1., wspace=0., hspace=0.)

# panel A (placeholder for svg figure to be inserted; see below)
panel_label_pos_shift = (-0.26, 0.5)
plt.subplot(gs[0, 0])
plt.axis('off')

# panel C (placeholder for svg figure to be inserted; see below)
plt.subplot(gs[2, 0])
plt.axis('off')

# panel C (placeholder for svg figure to be inserted; see below)
plt.subplot(gs[4, 0])
plt.axis('off')

for j, (learn, seq_num) in enumerate(zip([False, True, True], [0, 0, 1])):

    ###################################
    # postprocessing of data
    # ---------------------------------
    if learn and seq_num == 0:
        start_time = characters_to_time_excitation[sequences[0][0]][-1] - params['pad_time']
        end_time = characters_to_time_excitation[sequences[0][-1]][-1] + params['pad_time']
    elif learn and seq_num == 1:
        start_time = characters_to_time_excitation[sequences[1][0]][-1] - params['pad_time']
        end_time = characters_to_time_excitation[sequences[1][-1]][-1] + params['pad_time']
    else:
        start_time = characters_to_time_excitation[sequences[0][0]][0] - params['pad_time']
        end_time = characters_to_time_excitation[sequences[0][-1]][0] + params['pad_time']

    # select data  corresponding to the different sequences
    idx_somatic_spikes = np.where((somatic_spikes[:,1] > start_time) & (somatic_spikes[:,1] < end_time))
    idx_dAP = np.where((dendriticAP_times > start_time) & (dendriticAP_times < end_time))

    # postprocess somatic spikes
    somatic_spikes_times = somatic_spikes[:,1][idx_somatic_spikes]
    somatic_spikes_senders = somatic_spikes[:,0][idx_somatic_spikes]
    initial_time = somatic_spikes_times[0]
    somatic_spikes_times -= initial_time
    xmax = somatic_spikes_times[-1] + delta_time

    # postporcess dendritic AP
    dAP_senders = dendriticAP_senders[idx_dAP]
    dAP_currents = dendriticAP_currents[idx_dAP]
    dAP_times = dendriticAP_times[idx_dAP]
    dAP_times -= initial_time

    idx_exc_times = np.where((excitation_times > start_time) & (excitation_times < end_time))
    excitation_times_sel = excitation_times[idx_exc_times]

    # ###############################
    # draw stimulus
    # -------------------------------
    plt.subplot(gs[2*j, 1])
    plt.axis('off')

    for i in range(len(sequences[seq_num])): 

        x = (excitation_times_sel[i]+delta_time-initial_time) / (xmax+delta_time)
        y = 0.26
        arrow_width = 0.03
        arrow_height = 0.2

        pos = [x, y]
        X = np.array([pos, [pos[0]+arrow_width, pos[1]], [pos[0]+arrow_width/2, pos[1]-arrow_height]])
        t1 = plt.Polygon(X, color='black')
        plt.gca().add_patch(t1)
        #plt.text(pos[0]+arrow_width/8, pos[1]+0.5, sequences[seq_num][i])
        plt.text(pos[0]-0.003, pos[1]+0.1, sequences[seq_num][i])

    # ###############################
    # show soma and dendritic spikes
    # -------------------------------  
    plt.subplot(gs[2*j+1, 1])

    senders_subsampled = somatic_spikes_senders[::fraction_active]
    line1 = plt.plot(somatic_spikes_times[::fraction_active], somatic_spikes_senders[::fraction_active], 'o', color=color_soma_spike, lw=0., ms=0.5, zorder=2)

    #for k,v in count_indices_ds.items():
    for sender in senders_subsampled:
        idx_sub = np.where(dAP_senders == sender)
        line2 = plt.plot(dAP_times[idx_sub], dAP_senders[idx_sub], color=color_dendrite_spike, lw=1., zorder=1)

    plt.xlim(-delta_time, xmax)
    plt.ylim(-10, num_neurons+10)

    ticks_pos = shifted_subpopulation_indices * params['n_E']
    ticks_label = chars_per_subpopulation
    subpopulation_indices_background = np.arange(params['M'])*params['n_E']

    plt.yticks(ticks_pos, ticks_label)
    plt.tick_params(labelbottom=False)

    for i in range(params['M'])[::2]:
        plt.axhspan(subpopulation_indices_background[i], subpopulation_indices_background[i]+params['n_E'], facecolor=fc_bg, zorder=0)

    if j == 2:
        plt.xlabel('time (ms)')
        plt.tick_params(labelbottom=True)

    if j == 0:
        labels = ['somatic spikes', 'dendritic AP']
        plt.legend((line1[0], line2[0]), labels)

plt.savefig('/tmp/%s.png' % (master_file_name))

## Figure 1

In [ ]:
p = para.ParameterSpace({})

DELAY = 0.1

p['dt'] = 0.1                                  # simulation time resolution (ms)
p['print_simulation_progress'] = False         # print the time progress.

# neuron parameters of the excitatory neurons
p['soma_model'] = neuron_model_name
p['soma_params'] = {}
p['soma_params']['C_m'] = 250.        # membrane capacitance (pF)
p['soma_params']['E_L'] = 0.          # resting membrane potential (mV)
# p['soma_params']['I_e'] = 0.        # external DC currents (pA)
p['soma_params']['V_m'] = 0.          # initial potential (mV)
p['soma_params']['V_reset'] = 0.      # reset potential (mV)
p['soma_params']['V_th'] = 20.        # spike threshold (mV)
p['soma_params']['t_ref'] = 10.       # refractory period
p['soma_params']['tau_m'] = 10.       # membrane time constant (ms)
p['soma_params']['tau_syn1'] = 2.     # synaptic time constant: external input (receptor 1)
p['soma_params']['tau_syn2'] = 5.     # synaptic time constant: dendrtic input (receptor 2)
p['soma_params']['tau_syn3'] = 1.     # synaptic time constant: inhibitory input (receptor 3)
# dendritic action potential
p['soma_params']['I_p'] = 200. # current clamp value for I_dAP during a dendritic action potenti
p['soma_params']['tau_dAP'] = 60.       # time window over which the dendritic current clamp is active
p['soma_params']['theta_dAP'] = 59.        # current threshold for a dendritic action potential

p['soma_params']['I_dend_incr'] = 2.71 / 10e-3


p['fixed_somatic_delay'] = 2          # this is an approximate time of how long it takes the soma to fire
                                      # upon receiving an external stimulus 

# neuron parameters for the inhibitory neuron
p['inhibit_model'] = 'iaf_psc_exp'
p['inhibit_params'] = {}
p['inhibit_params']['C_m'] = 250.         # membrane capacitance (pF)
p['inhibit_params']['E_L'] = 0.           # resting membrane potential (mV)
p['inhibit_params']['I_e'] = 0.           # external DC currents (pA)
p['inhibit_params']['V_m'] = 0.           # initial potential (mV)
p['inhibit_params']['V_reset'] = 0.       # reset potential (mV)
p['inhibit_params']['V_th'] = 15.         # spike threshold (mV)
p['inhibit_params']['t_ref'] = 2.0        # refractory period
p['inhibit_params']['tau_m'] = 5.         # membrane time constant (ms)
p['inhibit_params']['tau_syn_ex'] = .5    # synaptic time constant of an excitatory input (ms) 
p['inhibit_params']['tau_syn_in'] = 1.65  # synaptic time constant of an inhibitory input (ms)

# synaptic parameters
p['J_EX_psp'] = 1.1 * p['soma_params']['V_th']     # somatic PSP as a response to an external input
p['J_IE_psp'] = 1.2 * p['inhibit_params']['V_th']  # inhibitory PSP as a response to an input from E neuron
p['J_EI_psp'] = -2 * p['soma_params']['V_th']      # somatic PSP as a response to an inhibitory input
p['convergence'] = 5
p['pattern_size'] = 20

# parameters for ee synapses (stdsp)
p['syn_dict_ee'] = {}
p['permanence_min'] = 0.
p['permanence_max'] = 8.
p['calibration'] = 40.
p['syn_dict_ee']['weight'] = 0.01                    # synaptic weight
p['syn_dict_ee']['synapse_model'] = synapse_model_name  # synapse model
p['syn_dict_ee']['permanence_threshold'] = 10.                    # synapse maturity threshold
p['syn_dict_ee']['tau_pre_trace'] = 20.                   # plasticity time constant (potentiation)
p['syn_dict_ee']['delay'] = 2.                       # dendritic delay 
p['syn_dict_ee']['receptor_type'] = 2                # receptor corresponding to the dendritic input
p['syn_dict_ee']['lambda_plus'] = 0.05 #0.1                     # potentiation rate
p['syn_dict_ee']['zt'] = 1.                          # target dAP trace [pA]
p['syn_dict_ee']['lambda_h'] = 0.01                        # homeostasis rate
p['syn_dict_ee']['Wmax'] = 1.1 * p['soma_params']['theta_dAP'] / p['convergence']   # Maximum allowed weight
p['syn_dict_ee']['permanence_max'] = 20.                       # Maximum allowed permanence
p['syn_dict_ee']['permanence_min'] = 1.                        # Minimum allowed permanence
p['syn_dict_ee']['lambda_minus'] = 0.004

# parameters of EX synapses (external to soma of E neurons)
p['conn_dict_ex'] = {}
p['syn_dict_ex'] = {}
p['syn_dict_ex']['receptor_type'] = 1                    # receptor corresponding to external input
p['syn_dict_ex']['delay'] = DELAY                        # dendritic delay
p['conn_dict_ex']['rule'] = 'all_to_all'                 # connection rule

# parameters of EdX synapses (external to dendrite of E neurons) 
p['conn_dict_edx'] = {}
p['syn_dict_edx'] = {}
p['syn_dict_edx']['receptor_type'] = 2                    # receptor corresponding to the dendritic input
p['syn_dict_edx']['delay'] = DELAY                        # dendritic delay
p['syn_dict_edx']['weight'] = 10.4 * p['soma_params']['theta_dAP']
p['conn_dict_edx']['rule'] = 'fixed_outdegree'            # connection rule
p['conn_dict_edx']['outdegree'] = p['pattern_size'] + 1   # outdegree

# parameters for IE synapses 
p['syn_dict_ie'] = {}
p['conn_dict_ie'] = {}
p['syn_dict_ie']['synapse_model'] = 'static_synapse'     # synapse model
p['syn_dict_ie']['delay'] = DELAY                        # dendritic delay
p['conn_dict_ie']['rule'] = 'fixed_indegree'             # connection rule
p['conn_dict_ie']['indegree'] = 5                        # indegree 

# parameters for EI synapses
p['syn_dict_ei'] = {}
p['conn_dict_ei'] = {}
p['syn_dict_ei']['synapse_model'] = 'static_synapse'     # synapse model
p['syn_dict_ei']['delay'] = DELAY                        # dendritic delay
p['syn_dict_ei']['receptor_type'] = 3                    # receptor corresponding to the inhibitory input  
p['conn_dict_ei']['rule'] = 'fixed_indegree'             # connection rule
p['conn_dict_ei']['indegree'] = 20                       # indegree

In [ ]:
print("Running simulations")
data = {}
for i, name in enumerate(['ff', 'dendrite', 'ff_dendrite']): 
    print("Running experiment type: " + name)
    # init kernel
    seed = 1
    nest.ResetKernel()
    nest.set_verbosity("M_ALL")
    nest.SetKernelStatus({
        'resolution': params['dt'],
        'print_time': params['print_simulation_progress'],
        #'local_num_threads': n_threads
        'rng_seed': seed
    })

    data[name] = {}

    #############################
    # create and connect neurons
    # ---------------------------

    # create excitatory population
    exc_neuron = nest.Create(params['soma_model'], params=params['soma_params'])

    # create inhibitory population
    inh_neuron = nest.Create(params['inhibit_model'], params=params['inhibit_params'])

    # connect inhibition
    nest.Connect(exc_neuron, inh_neuron, syn_spec=params['syn_dict_ie'])
    nest.Connect(inh_neuron, exc_neuron, syn_spec=params['syn_dict_ei'])

    ######################
    # Input stream/stimuli
    #---------------------
    input_excitation = nest.Create('spike_generator', params={'spike_times':[soma_excitation_time]})
    dendrite_excitation_1 = nest.Create('spike_generator', params={'spike_times':[dendrite_excitation_time]})
    #dendrite_excitation_2 = nest.Create('spike_generator', params={'spike_times':[7.]})
    inhibition_excitation = nest.Create('spike_generator', params={'spike_times':[10.]})

    # excitation soma feedforward
    if name == 'ff' or name == 'ff_dendrite':
        nest.Connect(input_excitation, exc_neuron, syn_spec={'receptor_type': 1, 
                                                             'weight': params['syn_dict_ex']['weight'], 
                                                             'delay': params['syn_dict_ex']['delay']})

    # excitation dendrite 
    if name == 'dendrite' or name == 'ff_dendrite':
        nest.Connect(dendrite_excitation_1, exc_neuron, syn_spec={'receptor_type': 2, 
                                                                  'weight': params['syn_dict_edx']['weight'], 
                                                                  'delay': params['syn_dict_edx']['delay']})

    # record voltage inhibitory neuron 
    vm_inh = nest.Create('voltmeter', params={'record_from': ['V_m'], 'interval': 0.1})
    nest.Connect(vm_inh, inh_neuron)

    # record voltage soma
    vm_exc = nest.Create('voltmeter', params={'record_from': ['V_m'], 'interval': 0.1})
    nest.Connect(vm_exc, exc_neuron)

    active_dendrite_exc_mm = nest.Create('multimeter', params={'record_from': ['active_dendrite_readout', 'I_dend'], 'interval': 0.1})
    nest.Connect(active_dendrite_exc_mm, exc_neuron)

    # record spikes
    sd = nest.Create('spike_recorder')
    nest.Connect(exc_neuron, sd)

    # record inh spikes
    sd_inh = nest.Create('spike_recorder')
    nest.Connect(inh_neuron, sd_inh)

    print('### simulating network')
    #nest.Simulate(100.)
    nest.Prepare()
    nest.Run(100.)

    voltage_soma = nest.GetStatus(vm_exc)[0]['events']  
    active_dendrite = nest.GetStatus(active_dendrite_exc_mm)[0]['events']
    voltage_inhibit = nest.GetStatus(vm_inh)[0]['events'] 
    spikes_soma = nest.GetStatus(sd)[0]['events'] 
    spikes_inh = nest.GetStatus(sd_inh)[0]['events'] 

    data[name]['exc'] = voltage_soma 
    data[name]['exc_active_dendrite'] = active_dendrite 
    data[name]['inh'] = voltage_inhibit
    data[name]['spikes_exc'] = spikes_soma
    data[name]['spikes_inh'] = spikes_inh

## Plotting

In [ ]:
def position_excitation_arrows(ax, soma_time, dendrite_time):

    arrow_width = 1.8
    arrow_height = 1.8
    y = -2.3
    
    # plot excitation arrows for panel A
    x = soma_time - arrow_width/2 
    pos = [x, y]
    X = np.array([pos, [pos[0]+arrow_width, pos[1]], [pos[0]+arrow_width/2, pos[1]+arrow_height]])
    t1 = plt.Polygon(X, color=color_somatic_input)
    ax[0].add_patch(t1)

    # plot excitation arrows for panel B
    x = dendrite_time - arrow_width/2 
    pos = [x, y]
    X = np.array([pos, [pos[0]+arrow_width, pos[1]], [pos[0]+arrow_width/2, pos[1]+arrow_height]])
    t1 = plt.Polygon(X, color=color_dAP_input)
    ax[1].add_patch(t1)

    # plot excitation arrows for panel C
    x = dendrite_time - arrow_width/2 
    pos = [x, y]
    X = np.array([pos, [pos[0]+arrow_width, pos[1]], [pos[0]+arrow_width/2, pos[1]+arrow_height]])
    t1 = plt.Polygon(X, color=color_dAP_input)
    ax[2].add_patch(t1)

    x = soma_time - arrow_width/2 
    pos = [x, y]
    X = np.array([pos, [pos[0]+arrow_width, pos[1]], [pos[0]+arrow_width/2, pos[1]+arrow_height]])
    t1 = plt.Polygon(X, color=color_somatic_input)
    ax[2].add_patch(t1)


color_dAP_input = '#8e7c42ff'
#color_somatic_input = '#0000ffff'
color_somatic_input = '#4581a7ff'
color_soma = '#000000ff'
color_dAP = '#00B4BE' 
color_inhibit = '#808080ff'  
color_hrl = 'black'

#color_somatic_spike = '#ff0000ff'
color_somatic_spike = color_soma
color_inh_spike = color_inhibit
ms_spike = 7
mew_spike = 1.5
lw_vtheta = 0.5
lw_dAP = 1.5
lw_s = 1.5
lw_i = 1.5

# plot settings 
fig_size = (6., 5)
ymin = -4
ymax = params['soma_params']['V_th'] + 4
xmin = 0  
xmax = 85
label_pos = (-0.18, 1.)
panel_labels = ['A', 'B', 'C']
v_th=params['soma_params']['V_th'] 
time_dAP = 10
soma_excitation_time = 25.
dendrite_excitation_time = 3.

# set up the figure frame
fig = plt.figure()
gs = mpl.gridspec.GridSpec(5, 1, height_ratios=[15,15,15,5,6], bottom=0.1, right=0.95, top=0.93, wspace=0., hspace=0.1)
left, bottom, width, height = [0.4, 0.1, 0.2, 0.2]
axes = []



for i, name in enumerate(['ff', 'dendrite', 'ff_dendrite']):

    #ax = fig.add_subplot(gs[i,0])
    ax = plt.subplot(gs[i,0])
    ax.text(label_pos[0], label_pos[1], panel_labels[i], transform=ax.transAxes, horizontalalignment='center', verticalalignment='center', size=10, weight='bold')
    ax.plot(data[name]['exc']['times'], data[name]['exc']['V_m'], lw=lw_s, color=color_soma, zorder=2, label='excitatory neuron')  
    
    
    ax.plot(data[name]['exc_active_dendrite']['times'], data[name]['exc_active_dendrite']['active_dendrite_readout'], lw=lw_s, color=color_dAP)  
    
    ax_ = ax.twinx()
    ax_.plot(data[name]['exc_active_dendrite']['times'], data[name]['exc_active_dendrite']['I_dend'], lw=lw_s, color="red", label="I_dend")  
    ax_.plot((0., np.amax(data[name]['exc_active_dendrite']['times'])), 2*[p['soma_params']['theta_dAP']], c="red", linestyle=':')
             
    ax.plot(data[name]['spikes_exc']['times'], (v_th+2)*np.ones(len(data[name]['spikes_exc']['times'])), '|', c=color_somatic_spike, ms=ms_spike, mew=mew_spike)
    ax.plot(data[name]['spikes_inh']['times'], (v_th+2)*np.ones(len(data[name]['spikes_inh']['times'])), '|', c=color_inh_spike, ms=ms_spike, mew=mew_spike)
    ax.legend()
 
    # add dendritic action potential bar manually
    if name == 'dendrite': 
        ax.hlines(v_th+2, time_dAP, time_dAP+params['soma_params']['tau_dAP'], lw=lw_dAP, color=color_dAP)

    if name == 'ff_dendrite': 
        ax.hlines(v_th+2, time_dAP, data[name]['spikes_exc']['times'][0], lw=lw_dAP, color=color_dAP)

    # clamp voltage if doesn't reach the firing threshold
    if name == 'ff' or name == 'ff_dendrite': 
        max_volt = max(data[name]['inh']['V_m']) 
        max_volt_ind = np.where(data[name]['inh']['V_m']==max_volt)[0]
        data[name]['inh']['V_m'][max_volt_ind] = 20

    ax.plot(data[name]['inh']['times'], data[name]['inh']['V_m'], lw=lw_i, color=color_inhibit, zorder=1, label='inhibitory neuron') 
    ax.set_ylim([ymin, ymax])
    ax.set_xlim([xmin, xmax])
    ax.hlines(v_th, xmin, xmax, lw=lw_vtheta, color=color_hrl, linestyle='--')

    axes.append(ax)

axes[1].set_ylabel('membrane potential (mV)')

# set position of arrows
position_excitation_arrows(axes, soma_excitation_time, dendrite_excitation_time)

axes[0].legend(loc='center right')
axes[0].set_yticklabels([])
axes[0].set_xticklabels([])
axes[1].set_xticklabels([])
axes[2].set_yticklabels([])
axes[2].set_xlabel('time (ms)')

########################################
# plt spikes of A and B
# --------------------------------------
ax = fig.add_subplot(gs[i+1,0])
plt.axis('off')

ax = plt.subplot(gs[i+2,0])
ax.text(label_pos[0], label_pos[1], 'D', transform=ax.transAxes, horizontalalignment='center', verticalalignment='center', size=10, weight='bold')

xmin_d=25.6
xmax_d=29

ymin_d=0
ymax_d=10

name = 'ff'
ax.plot(data[name]['spikes_exc']['times'], (3*ymax_d/4)*np.ones(len(data[name]['spikes_exc']['times'])), '|', c=color_somatic_spike, ms=ms_spike, mew=mew_spike)
ax.plot(data[name]['spikes_inh']['times'], (3*ymax_d/4)*np.ones(len(data[name]['spikes_inh']['times'])), '|', c=color_inh_spike, ms=ms_spike, mew=mew_spike)

name = 'ff_dendrite'
ax.plot(data[name]['spikes_exc']['times'], (ymax_d/4)*np.ones(len(data[name]['spikes_exc']['times'])), '|', c=color_somatic_spike, ms=ms_spike, mew=mew_spike)
ax.plot(data[name]['spikes_inh']['times'], (ymax_d/4)*np.ones(len(data[name]['spikes_inh']['times'])), '|', c=color_inh_spike, ms=ms_spike, mew=mew_spike)
ax.hlines(ymax_d/2, xmin, xmax, lw=0.5, color=color_hrl, linestyles='solid')

ax.set_yticklabels([])
ax.tick_params(left=False)
ax.set_ylim([ymin_d, ymax_d])
ax.set_xlim([xmin_d, xmax_d])
ax.set_xlabel('time (ms)')

ax.text(xmin_d+0.05, (3*ymax_d/4)-1, 'A', size=8, weight='bold')
ax.text(xmin_d+0.05, (ymax_d/4)-1, 'C', size=8, weight='bold')

############################################################
# add lines between the subplots showing the zoomed in area
# ----------------------------------------------------------
xy_C = (xmin_d,ymin)
xy_D = (xmin_d,ymax_d)
con = mpl.patches.ConnectionPatch(xyA=xy_C, xyB=xy_D, coordsA='data', coordsB='data', axesA=axes[-1], axesB=ax, color='grey', linestyle='dotted')
ax.add_artist(con)

xy_C = (xmax_d,ymin)
xy_D = (xmax_d,ymax_d)
con = mpl.patches.ConnectionPatch(xyA=xy_C, xyB=xy_D, coordsA='data', coordsB='data', axesA=axes[-1], axesB=ax, color='grey', linestyle='dotted')
ax.add_artist(con)

plt.savefig("/tmp/sequences1.png")

## Plasticity dynamics dependence on dAP

In [ ]:

DELAY = 0.1

p = para.ParameterSpace({})

p['dt'] = 0.1                                  # simulation time resolution (ms)
p['print_simulation_progress'] = True         # print the time progress.

# neuron parameters of the excitatory neurons
p['soma_model'] = neuron_model_name
p['soma_params'] = {}
p['soma_params']['C_m'] = 250.        # membrane capacitance (pF)
p['soma_params']['E_L'] = 0.          # resting membrane potential (mV)
# p['soma_params']['I_e'] = 0.        # external DC currents (pA)
p['soma_params']['V_m'] = 0.          # initial potential (mV)
p['soma_params']['V_reset'] = 0.      # reset potential (mV)
p['soma_params']['V_th'] = 20.        # spike threshold (mV)
p['soma_params']['t_ref'] = 10.       # refractory period
p['soma_params']['tau_m'] = 10.       # membrane time constant (ms)
p['soma_params']['tau_syn1'] = 2.     # synaptic time constant: external input (receptor 1)
p['soma_params']['tau_syn2'] = 5.     # synaptic time constant: dendrtic input (receptor 2)
p['soma_params']['tau_syn3'] = 1.     # synaptic time constant: inhibitory input (receptor 3)
# dendritic action potential
p['soma_params']['I_p'] = 200. # current clamp value for I_dAP during a dendritic action potenti
p['soma_params']['tau_dAP'] = 60.       # time window over which the dendritic current clamp is active
p['soma_params']['theta_dAP'] = 59.        # current threshold for a dendritic action potential
p['fixed_somatic_delay'] = 2          # this is an approximate time of how long it takes the soma to fire
                                      # upon receiving an external stimulus 

# synaptic parameters
p['J_EX_psp'] = 1.1 * p['soma_params']['V_th']     # somatic PSP as a response to an external input
p['convergence'] = 5
p['pattern_size'] = 20       # sparse set of active neurons per subpopulation

# parameters for ee synapses (stdsp)
p['syn_dict_ee'] = {}
p['permanence_min'] = 0.
p['permanence_max'] = 8.
p['calibration'] = 0.
p['syn_dict_ee']['weight'] = 0.01                    # synaptic weight
p['syn_dict_ee']['synapse_model'] = synapse_model_name  # synapse model
p['syn_dict_ee']['permanence_threshold'] = 10.                    # synapse maturity threshold
p['syn_dict_ee']['tau_pre_trace'] = 20.                   # plasticity time constant (potentiation)
p['syn_dict_ee']['delay'] = 2.                       # dendritic delay 
p['syn_dict_ee']['receptor_type'] = 2                # receptor corresponding to the dendritic input
p['syn_dict_ee']['lambda_plus'] = 0.08                     # potentiation rate
p['syn_dict_ee']['zt'] = 1.                          # target dAP trace [pA]
p['syn_dict_ee']['lambda_h'] = 0.014                        # homeostasis rate
p['syn_dict_ee']['Wmax'] = 1.1 * p['soma_params']['theta_dAP'] / p['convergence']   # Maximum allowed weight
p['syn_dict_ee']['permanence_max'] = 20.                       # Maximum allowed permanence
p['syn_dict_ee']['permanence_min'] = 1.                        # Minimum allowed permanence
p['syn_dict_ee']['lambda_minus'] = 0.0015

# parameters of EX synapses (external to soma of E neurons)
p['conn_dict_ex'] = {}
p['syn_dict_ex'] = {}
p['syn_dict_ex']['receptor_type'] = 1                    # receptor corresponding to external input
p['syn_dict_ex']['delay'] = DELAY                        # dendritic delay
p['conn_dict_ex']['rule'] = 'all_to_all'                 # connection rule

## stimulus parameters
p['DeltaT'] = 40.                               # inter-stimulus interval

In [ ]:
nest.ResetKernel()
nest.set_verbosity("M_ALL")
nest.SetKernelStatus({
    'resolution': params['dt'],
    'print_time': params['print_simulation_progress'],
    #'local_num_threads': n_threads,
    'rng_seed': seed
})

In [ ]:
# neuron parameters
params = p

neuron_1 = nest.Create(params['soma_model'], params=params['soma_params'])
neuron_2 = nest.Create(params['soma_model'], params=params['soma_params'])

# connect two neurons
nest.Connect(neuron_1, neuron_2, syn_spec=params['syn_dict_ee'])

# creation of spike generator
time_neuron_1 = 10.
time_neuron_2 = time_neuron_1 + params['DeltaT']

training_steps = 120
between_exc = 5*params['DeltaT']

times_neuron_1 = [time_neuron_1+i*between_exc for i in range(training_steps)]
times_neuron_2 = [time_neuron_2+i*between_exc for i in range(training_steps)]#[:10]

# create the spike generators 
# disable spike generator for the interval 'dis', to see the affect of stpd
dis = 20
spike_generator_1 = nest.Create('spike_generator', params={'spike_times': times_neuron_1})
spike_generator_2 = nest.Create('spike_generator', params={'spike_times': times_neuron_2})

# connect the spike generator 

params['R_m_soma'] = params['soma_params']['tau_m'] / params['soma_params']['C_m']
params['syn_dict_ex']['weight'] = psp_max_2_psc_max(params['J_EX_psp'], 
                                                           params['soma_params']['tau_m'], 
                                                           params['soma_params']['tau_syn1'], 
                                                           params['R_m_soma'])

syn_dict_ff = {'receptor_type': 1, 'weight': params['syn_dict_ex']['weight'], 'delay': params['syn_dict_ex']['delay']}
nest.Connect(spike_generator_1, neuron_1, syn_spec=syn_dict_ff)
nest.Connect(spike_generator_2, neuron_2, syn_spec=syn_dict_ff)

# record voltage neuron 1, neuron 2
dap_mm_1 = nest.Create('multimeter', {"record_from": ["dAP_trace"]})
nest.Connect(dap_mm_1, neuron_1)

dap_mm_2 = nest.Create('multimeter', {"record_from": ["dAP_trace"]})
nest.Connect(dap_mm_2, neuron_2)

vm_1 = nest.Create('voltmeter')
vm_2 = nest.Create('voltmeter')
nest.Connect(vm_1, neuron_1)
nest.Connect(vm_2, neuron_2)


sd_1 = nest.Create('spike_recorder')
nest.Connect(neuron_1, sd_1)

sd_2 = nest.Create('spike_recorder')
nest.Connect(neuron_2, sd_2)


synColl = nest.GetConnections(synapse_model=synapse_model_name)
assert len(synColl) == 1

print('\n simulate')
zs = [0.,1.,2.]
weights_cs = []
permanences_cs = []

for z in zs:

    weights = []
    permanences = []
    last_sim_time = 0
    
    spike_generator_1.origin = nest.GetKernelStatus('biological_time')
    spike_generator_2.origin = nest.GetKernelStatus('biological_time')
    
    # connect two neurons
    synColl.set({'permanence': 1.}) 

    for i in range(training_steps):

        # change toward using the weight recorder, example:
        #wr = nest.Create('weight_recorder')
        #nest.CopyModel('stdp_synapse', 'stdp_synapse_rec', {'weight_recorder': wr})

        nest.SetStatus(neuron_1, {'dAP_trace':z})
        nest.SetStatus(neuron_2, {'dAP_trace':z})

        # simulate the network
        sim_time = times_neuron_1[i] - last_sim_time 
        nest.Simulate(sim_time)
        last_sim_time = times_neuron_1[i]

        w_after = synColl.weight
        p_after = synColl.permanence
        weights.append(w_after)
        permanences.append(p_after)

        
    fig, ax = plt.subplots(figsize=(24,6))
    ax.plot(nest.GetStatus(vm_1)[0]['events']["times"], nest.GetStatus(vm_1)[0]['events']["V_m"], label="vm1")
    max_V_m = np.amax(nest.GetStatus(vm_1)[0]['events']["V_m"])
    ax.scatter(nest.GetStatus(sd_1)[0]['events']['times'], max_V_m * np.ones_like(nest.GetStatus(sd_1)[0]['events']['times']))
    ax.plot(nest.GetStatus(vm_2)[0]['events']["times"], nest.GetStatus(vm_2)[0]['events']["V_m"], label="vm2")
    ax.scatter(nest.GetStatus(sd_2)[0]['events']['times'], max_V_m * np.ones_like(nest.GetStatus(sd_2)[0]['events']['times']))
    ax.legend()
    ax_ = ax.twinx()
    ax_.plot(nest.GetStatus(dap_mm_1)[0]['events']["times"], nest.GetStatus(dap_mm_1)[0]['events']["dAP_trace"], label="dAP")
    ax_.plot(nest.GetStatus(dap_mm_2)[0]['events']["times"], nest.GetStatus(dap_mm_2)[0]['events']["dAP_trace"], label="dAP")
    
    fig.savefig("/tmp/foo" + str(z) + ".png")
        
    weights_cs.append(weights)
    permanences_cs.append(permanences)

# store postprocessed
params['zs'] = zs


In [ ]:
# load data
# ----------

path_dict = {} 
path_dict['data_root_path'] = 'data'
path_dict['project_name'] = 'sequence_learning_performance' 
path_dict['parameterspace_label'] = 'plasticity_dynamics'


# plot recorded data
# ------------------

# plot settings 
fig_size = (5.2, 2.)
plt.rcParams['font.size'] = 8
plt.rcParams['legend.fontsize'] = 6
plt.rcParams['figure.figsize'] = fig_size
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['text.usetex'] = False
ms = 0.5
alpha = 0.5
lw_hline = 1.

#################
# visualize data
# ---------------
gs = mpl.gridspec.GridSpec(1, 3, right=0.92, left=0.09, bottom=0.2, top=0.89, wspace=0.2, hspace=0.2)

# data for Ic=0
# -------------
ax1 = plt.subplot(gs[0,0])

training_steps = len(weights_cs[0])
num_pulses = np.arange(training_steps)
lns1 = ax1.plot(num_pulses, weights_cs[0], '-o', ms=ms, color='black', label=r'$J$')

#plt.ylabel('weight ($\mu$S)')
ax1.set_xlim(0, training_steps)
ax1.set_ylim(-1, params["syn_dict_ee"]['Wmax']+10)
#ax1.set_title(r'dAP rate $\nu_\mathsf{d}$=%0.1f' % zs[0])
ax1.set_title(r'$z$=%0.1f' % zs[0])
ax1.set_ylabel(r'weight $J$ (pA)')

ax2 = ax1.twinx()
lns2 = ax2.plot(num_pulses, permanences_cs[0], '-o', ms=ms, color='grey', alpha=alpha, label=r'$P$')
plt.hlines(params['syn_dict_ee']['permanence_threshold'], 0, training_steps, lw=lw_hline, color='grey', linestyles='dotted')
plt.hlines(params['syn_dict_ee']['permanence_max'], 0, training_steps, lw=lw_hline, color='grey', linestyles='dashed')

ax2.set_ylim(-1, params["syn_dict_ee"]['permanence_max']+2)
ax2.tick_params(axis='y', labelcolor='grey')
#ax2.set_yticklabels([])
ax2.set_yticks([])
ax2.spines['right'].set_color('grey')

# add legends
lns = [lns1[0],lns2[0]]
labs = [l.get_label() for l in lns]
ax1.legend(lns, labs, loc='lower right')

# data for Ic=1
# -------------
ax1 = plt.subplot(gs[0,1])

ax1.plot(num_pulses, weights_cs[1], '-o', ms=ms, color='black', label='weight')

ax1.set_ylim(-1, params["syn_dict_ee"]['Wmax']+10)
ax1.set_xlim(0, training_steps)
#ax1.set_title(r'dAP rate $\nu_\mathsf{d}$=%0.1f' % zs[1])
ax1.set_title(r'$z$=%0.1f' % zs[1])
ax1.set_xlabel('number of presynaptic-postsynaptic spike pairings')
ax1.set_yticks([])

ax2 = ax1.twinx()
ax2.plot(num_pulses, permanences_cs[1], '-o', ms=ms, color='grey', alpha=alpha, label='permanence')
plt.hlines(params['syn_dict_ee']['permanence_threshold'], 0, training_steps, lw=lw_hline, color='grey', linestyles='dotted')
plt.hlines(params['syn_dict_ee']['permanence_max'], 0, training_steps, lw=lw_hline, color='grey', linestyles='dashed')

ax2.set_ylim(-1, params["syn_dict_ee"]['permanence_max']+2)
ax2.tick_params(axis='y', labelcolor='grey')
ax2.set_yticks([])
ax2.spines['right'].set_color('grey')

# data for Ic=2
# -------------
ax1 = plt.subplot(gs[0,2])

ax1.plot(num_pulses, weights_cs[2], '-o', ms=ms, color='black', label='weight')

ax1.set_ylim(-1, params["syn_dict_ee"]['Wmax']+10)
ax1.set_xlim(0, training_steps)
#ax1.set_title(r'dAP rate $\nu_\mathsf{d}$=%0.1f' % zs[2])
ax1.set_title(r'$z$=%0.1f' % zs[2])
ax1.set_yticks([])

ax2 = ax1.twinx()
ax2.plot(num_pulses, permanences_cs[2], '-o', ms=ms, color='grey', alpha=alpha, label=r'$P$')
plt.hlines(params['syn_dict_ee']['permanence_threshold'], 0, training_steps, lw=lw_hline, color='grey', linestyles='dotted')
plt.hlines(params['syn_dict_ee']['permanence_max'], 0, training_steps, lw=lw_hline, color='grey', linestyles='dashed')

ax2.set_ylim(-1, params["syn_dict_ee"]['permanence_max']+2)
ax2.tick_params(axis='y', labelcolor='grey')
ax2.set_ylabel(r"permanence $P$", color="grey")
#ax2.spines['right'].set_color('grey')

print('---------------------------------')
path = '.'
fname = 'plasticity_dynamics'
#print("save %s/%s.pdf" % (path, fname))
#plt.savefig("/tmp/%s.pdf" % fname)
plt.savefig("/tmp/%s.png" % fname)



References
----------

[1] Bouhadjar Y, Wouters DJ, Diesmann M, Tetzlaff T (2022) Sequence learning, prediction, and replay in networks of spiking neurons. PLoS Comput Biol 18(6): e1010233. https://doi.org/10.1371/journal.pcbi.1010233

